In [132]:
from itertools import islice, cycle
from tau_profile_parser import TauProfileParser
import pandas as pd
import plotly.graph_objs as go
import plotly as py
import plotly.express as px

Turn on offline mode

In [133]:
py.offline.init_notebook_mode(connected=True)

Plotly Bar Plot Examples
========================

This notebook demonstrates using the TauProfileParser to parse TAU profile.x.y.z files and using Plotly to generate ParaProf-style bar plots from them.

The TAU Profile Parser
----------------------

First, we call `TauProfileParser.parse` on the path to a directory containing TAU profile files. 

Note that the parser does not currently have any native support for MULTI_ directories, so if you have those
you'll have to call parse multiple times and then, if you wish, merge the resulting dataframes. This functionality should be added to the parser.

In [134]:
lulesh_data = TauProfileParser.parse('lulesh_profile')

This returns an object that provides access to the metric name, metadata, and the interval and atomic/userevent data.

The `metric` attribute contains the metric that values in this profile represent. (This representation will need to change when multi-metric data can be read in directly)


In [135]:
lulesh_data.metric

b'TIME'

The `metadata` attribute contains a Python dictionary of the metadata recorded in the profile.0.0.0 file

The `indices` attribute contains the (node, context, thread) tuples that are present in this profile.

## HeatMap

A heat map of the `col_name` values amongs the `TOP_N` common timers between all of the threads

In [136]:
node, context = 0, 0
col_name = 'Exclusive'
TOP_N = 10

In [137]:
def create_func_data(data):
    """
    Everything but the TAU_CALLPATH group
    
    param:
        data: DataFrame
    """
    data = data.loc[~data["Group"].str.contains("TAU_CALLPATH")]
    return data
    
def create_callpath_data(data):
    """
    Only the TAU_CALLPATH group
    
    param:
        data: DataFrame
    """
    data = data.loc[data["Group"].str.contains("TAU_CALLPATH")]
    return data
    

In [138]:
def find_num_threads(raw_data):
    """
    Finds number of threads found in raw_data
    
    param:
        raw_data: DataFrame (interval_data())
    """
    d = raw_data.loc[0, 0]
    threads = set([i[0] for i in d.index])
    return len(threads)

def trim_data(data, num_threads, top_n, col_name):
    """
    creates and returns a dataframe containing the `top_n` `col_name` column of the `data` DataFrame
    """
    
    df = None
    cntr = 0
    first = True
    for thread in range(num_threads):        
        tmp = data.loc[node, context, thread][[col_name]] 
        tmp = tmp.sort_values(by=[col_name], ascending=False)  
        if first: 
            df = tmp
        else: 
            df = df.merge(tmp, how='inner', on=['Timer'], suffixes=(f"_{str(cntr)}", f"_{str(cntr + 1)}"))
            cntr += 1
        first = False

    truncated = df[:top_n]
    truncated = truncated.fillna("")
    return truncated

In [139]:
num_threads = find_num_threads(lulesh_data.interval_data())
callpaths = create_callpath_data(lulesh_data.interval_data())
callpaths = trim_data(callpaths, num_threads, TOP_N, col_name)
data = create_func_data(lulesh_data.interval_data())
data = trim_data(data, num_threads, TOP_N, col_name)


In [140]:
def create_function_labels(data):
    """Creats a list of labels to be desplayed on the y axis of heatmap"""
    functions_labels = [f for f in data.index]
    return functions_labels

def trim_a_single_call(single_call):
    """Strips and gets rid of the arguments of the timers signature"""
    func_name, location, timer_type = TauProfileParser.extract_from_timer_name(single_call)
    func_name = func_name.strip() if func_name is not None else ""
    timer_type = f"[{timer_type.strip()}]" if timer_type is not None else ""
    single_call = " ".join([timer_type, func_name])
    single_call = single_call[:single_call.find("(")] if single_call.find("(") != -1 else single_call
    return single_call
    
def trim_single_call_data(single_calls):
    functions_labels = [trim_a_single_call(f) for f in single_calls]
    return functions_labels

def trim_call_path_data(call_paths):
    """A callpath contains individual calls separated by `=>`, this function trims each individual call"""
    functions_labels = []
    for f in call_paths:
        f = f.split(" => ")
        f = [trim_a_single_call(i) for i in f]
        label = " => ".join(f)
        functions_labels.append(label)
    return functions_labels

def create_thread_labels(num_threads):
    """Creates thread labels that are the x-axis labels of our heatmap"""
    return [f"thrd_{str(t)}" for t in range(num_threads)]

def create_heat_map_data(data): 
    """Creates a 2d list which is the `heat` data needed for the heat map"""
    heat_map_data = []
    for r in data.iterrows():
        row = [i for i in r[1]]
        heat_map_data.append(row)
    return heat_map_data

In [141]:
import plotly.graph_objects as go
def show_heat_map(heat_map_data, call_labels, thread_labels):
    fig = go.Figure(data=go.Heatmap(
                    z=heat_map_data,
                    x=thread_labels,
                    y=call_labels
                ))
    fig.show()
    
def create_labels(data, num_threads):
    """creates x-axis and y-axis labels"""
    thread_labels = create_thread_labels(num_threads)
    function_labels = create_function_labels(data)
    return thread_labels, function_labels

def show_heat_map_single_call(single_call_data, num_threads):
    """For single calls"""
    thread_labels ,function_labels = create_labels(single_call_data, num_threads)
    function_labels = trim_single_call_data(function_labels)
    heat_map_data = create_heat_map_data(single_call_data)
    show_heat_map(heat_map_data, function_labels, thread_labels)
    
def show_heat_map_call_path(call_path_data, num_threads):
    """For call_path"""
    thread_labels ,function_labels = create_labels(call_path_data, num_threads)
    function_labels = trim_call_path_data(function_labels)
    heat_map_data = create_heat_map_data(call_path_data)
    show_heat_map(heat_map_data, function_labels, thread_labels)
    

In [142]:

show_heat_map_single_call(data, num_threads)


In [143]:

show_heat_map_call_path(callpaths, num_threads)
